In [1]:
%matplotlib inline

from database.DataBaseModels import *

import networkx as nx
import json

In [2]:
author_graph = nx.MultiDiGraph()

In [3]:
for citation in db.Citation.select():
    for author_to in citation.article_id.article__author_article.author_id:
        for author_from in citation.cited_by.article__author_article.author_id:
            author_graph.add_edge(author_from, author_to)

In [6]:
print(len(author_graph.edges))
print(len(author_graph.nodes))

101121
5791


In [5]:
# Removing selfloops edges
author_graph.remove_edges_from([edge for edge in author_graph.selfloop_edges()])

In [7]:
# List of authors pairs who cited each other
result = []

for node in author_graph.nodes:
    for successor in author_graph.successors(node):
        if node in author_graph.successors(successor):
            result.append([successor, node])


for sublist in result:
    result.remove([sublist[1], sublist[0]])

In [8]:
json_file = open('authors_pairs.json', 'w')

json.dump({tup[0]:[tup[1][0].author_id, tup[1][1].author_id] for tup in enumerate(result)}, json_file)

In [25]:
both_side_citations_graph = nx.Graph()

both_side_citations_graph.add_edges_from([[author[0].name, author[1].name] for author in result])

In [15]:
len(result)

4861

In [95]:
from bokeh.io import show, output_notebook, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, WheelZoomTool, PanTool, ResetTool, LabelSet, ColumnDataSource
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

G = from_networkx(both_side_citations_graph, nx.spring_layout, scale = 5, center = (0, 0))

In [99]:
plot = Plot(plot_width = 800,
             plot_height = 800,
             x_range = Range1d(-5, 5),
             y_range = Range1d(-5, 5))

plot.title.text = "Both Side Citations"
G.node_renderer.glyph = Circle(size=15,
                               fill_color=Spectral4[0])

plot.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool(), WheelZoomTool(), PanTool(), ResetTool())


G.node_renderer.glyph = Circle(size=10, fill_color=Spectral4[3])
G.node_renderer.selection_glyph = Circle(size=10, fill_color=Spectral4[2])
G.node_renderer.hover_glyph = Circle(size=10, fill_color=Spectral4[1])

G.edge_renderer.glyph = MultiLine(line_color="#548251", line_width=2)
G.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=2)
G.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=2)

G.selection_policy = NodesAndLinkedEdges()
G.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(G)

#output_file("Both Side Authors Citations.html")
show(plot)